In [12]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchvision.datasets import MNIST
from torchvision import transforms
from torch.utils.data import DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device: ",device)

Using device:  cpu


In [17]:
transform = transforms.ToTensor()

train_data = MNIST(
    root = "data",
    train = True,
    download=True,
    transform = transform

)
test_data= MNIST(
    root= "data",
    train = False,
    download=True,
    transform= transform
)

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

class SimpleCNN(nn.Module):

    def __init__(self):
        super().__init__()

        self.features = nn.Sequential(
            nn.Conv2d(1,8,kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(8,16,kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
            
        )

        self.classifier = nn.Sequential(
            nn.Linear(16*7*7, 64),
            nn.ReLU(),
            nn.Linear(64,10)

        )

    def forward(self,x):
        x = self.features(x)
        x = x.view(x.size(0),-1)
        x = self.classifier(x)
        return x
    
model = SimpleCNN().to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=.001)

epochs = 10

for epoch in range(epochs):

    model.train()
    total_loss = 0

    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_fn(outputs,labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch { epoch +1}/{epochs} | loss: {total_loss:.4f}")

model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        predictions = outputs.argmax(dim = 1)

        correct += (predictions == labels).sum().item()
        total += labels.size(0)

accuracy = 100*correct/total 
print(f"Test accuracy: {accuracy:.2f}%")

torch.save(model.state_dict(), "Simple_CNN_MNIST.pth")
print("Model Saved")

Epoch 1/10 | loss: 313.8658
Epoch 2/10 | loss: 92.6798
Epoch 3/10 | loss: 65.1476
Epoch 4/10 | loss: 53.0883
Epoch 5/10 | loss: 43.8501
Epoch 6/10 | loss: 37.2056
Epoch 7/10 | loss: 32.1924
Epoch 8/10 | loss: 28.0257
Epoch 9/10 | loss: 24.6332
Epoch 10/10 | loss: 22.8173
Test accuracy: 98.72%
Model Saved
